In [ ]:
import csv
import ast

class Player:
    def __init__(self, nick):
        self.nick = nick
        self.elo = 1000
        self.wins = 0
        self.matches = 0
        self.player_id = self.nick


players_temp = {}
players = []

with open("matches.csv", newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)

    players_unique = set()

    for row in reader:
        players_a = row["a"]
        players_a = ast.literal_eval(players_a)

        players_b = row["b"]
        players_b = ast.literal_eval(players_b)

        for player in players_a:
            players_unique.add(player)

        for player in players_b:
            players_unique.add(player)
        
for player in players_unique:
    players_temp[player] = Player(player)


for player in players_temp:
    players.append(players_temp[player])


print("")
print(f"{len(players)} graczy")

In [ ]:
class MatchBasic:
    def __init__(self, match_id, winners, losers, date):
        self.match_id = match_id
        self.winners = winners
        self.losers = losers
        self.date = date

class MatchPlayer:
    def __init__(self, match_id, player_id, elo_change, is_winner):
        self.match_id = match_id
        self.player_id = player_id
        self.elo_change = elo_change
        self.is_winner = is_winner

class Match(MatchBasic):
    def __init__(self, match_id, winners, losers, date, match_players):
        super().__init__(match_id, winners, losers, date)
        self.match_players = match_players # list of MatchPlayer objects
        

with open("matches.csv", newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)

    matches = []

    for row in reader:
        match_id = row["id"]
        winners = []
        losers = []
        winner_team = row["winner"]

        if winner_team == "a":
            winners = ast.literal_eval(row["a"])
            losers = ast.literal_eval(row["b"])
        elif winner_team == "b":
            winners = ast.literal_eval(row["b"])
            losers = ast.literal_eval(row["a"])
        
        for idx,winner in enumerate(winners):
            for player in players:
                if player.nick == winner:
                    winners[idx] = player.player_id
        
        for idx,loser in enumerate(losers):
            for player in players:
                if player.nick == loser:
                    losers[idx] = player.player_id

        date = row["date"]
        
        match_players = []

        for player in winners:
            match_players.append(MatchPlayer(match_id, player, 0, True))
            
        for player in losers:
            match_players.append(MatchPlayer(match_id, player, 0, False))

        matches.append(Match(match_id, winners, losers, date, match_players))

for match in matches:
    print(f"{match.match_id}: {match.winners} vs {match.losers}")
    
print(f"{len(matches)} meczy")

In [ ]:
temp_players = players

for match in matches:
    for player in match.match_players:
        for temp_player in temp_players:
            if player.player_id == temp_player.player_id:
                player.elo_change = temp_player.elo
                if player.is_winner:
                    temp_player.wins += 1
                temp_player.matches += 1
